In [1]:
import time
import fluidsynth
import numpy as np
import glob

In [3]:
#### code from mike to do this in mido + pyfluidsynth
def mike_remapcoefficients(laguerre,minlog=0.,maxlog=4.):
    logcoefamplitudes = np.log10(np.abs(laguerre.coscoefs.T**2+laguerre.sincoefs.T**2))
    # override the max
    maxlog = np.nanmax(logcoefamplitudes)
    logcoefamplitudes = (logcoefamplitudes-minlog)/(maxlog-minlog)*127.
    logcoefamplitudes[logcoefamplitudes<0]=0
    return logcoefamplitudes


# major scale
scale_notes = np.array([60,62,64,65,67,69,71,72])


def make_note_list(logcoefamplitudes,scale_notes,octaveshift=True):

    notes = []
    for harmonic in range(0,5):
        if octaveshift:
            use_notes = scale_notes + (harmonic-1)*12
        else:
            use_notes = scale_notes
        newnote = [use_notes[x] for x in range(0,len(use_notes))]
        newnote = [(note,logcoefamplitudes[i, harmonic]) for i, note in enumerate(newnote) if logcoefamplitudes[i, harmonic] > 0]
        for note in newnote:
            notes.append(note)

    return notes

def add_rolled_chord(track, notes, duration, velocity=64):
    roll_spacing = 30  # in MIDI ticks (adjust for speed of roll)
    for i,note in enumerate(notes):
        track.append(Message('note_on', note=note[0], velocity=velocity, time=0 if i == 0 else roll_spacing))
    track.append(Message('note_off', note=notes[0][0], velocity=velocity, time=duration * 480))
    for note in notes[1:]:
        track.append(Message('note_off', note=note[0], velocity=velocity, time=0))

def add_rolled_chord_velocity(track, notes, duration):
    roll_spacing = 30  # in MIDI ticks (adjust for speed of roll)
    for i,note in enumerate(notes):
        track.append(Message('note_on', note=note[0], velocity=note[1], time=0 if i == 0 else roll_spacing))
    track.append(Message('note_off', note=notes[0][0], velocity=notes[0][1], time=duration * 480))
    for note in notes[1:]:
        track.append(Message('note_off', note=note[0], velocity=note[1], time=0))


def add_chord(track, notes, duration, velocity=64):
    for note in notes:
        track.append(Message('note_on', note=note[0], velocity=velocity, time=0))
    track.append(Message('note_off', note=notes[0][0], velocity=velocity, time=duration * 480))
    for note in notes[1:]:
        track.append(Message('note_off', note=note[0], velocity=velocity, time=0))

def make_midi_chords(lagcoefamplitudes,scale_notes,outputfile,octaveshift=False):

    # Create a new MIDI file and track
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set the tempo (optional, here 120 BPM)
    tempo = mido.bpm2tempo(120)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))

    # Insert program change at the beginning of the track
    program_change = mido.Message('program_change', program=4, time=0)  # Electric Piano 1
    track.insert(0, program_change)

    notes = make_note_list(logcoefamplitudes,scale_notes,octaveshift=octaveshift)

    #add_chord(track, notes, 4)
    add_rolled_chord(track, notes, 6)

        # Add a small gap before the next chord
        #track.append(Message('note_off', note=0, time=480))  # 240 ticks (half a beat)

    # Save the MIDI file
    mid.save(outputfile)

In [456]:
def mod_map_to_sounds_octave_ampcutoff(cos_cos, sin_cos):
    note_list = np.array(['D#','A#','F','C','G', 'D', 'A', 'E','B','F#','C#','G#'])
    #np.array(['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']) 
    mmax = cos_cos.shape[0]
    nmax = cos_cos.shape[1]

    modulus = np.sqrt(cos_cos**2 + sin_cos**2)
    
    abs_max = np.max(modulus)
    note_dict = {}
    
    for m in range(mmax):
        octave = m + 1 #hmm. need to bump from m = 0 as lowest to m = 1 bc m = 0 doesn't have all notes (?)
        for n in range(nmax):
            amp = modulus[m,n]
            if abs(amp) >= 0.01*abs_max: #will only play terms with amplitude at least 1% of max
                note = note_list[n] + str(octave)
                note_dict[note] = abs(amp)

    print('returning full set of notes')
    return note_dict

In [457]:
redo
len(np.unique(name_list)), len(name_list)
63/(316)

0.19936708860759494

In [458]:
redo = np.array([775, 780, 910, 915, 920, 925, 930, 935, 940, 945, 950, 955, 960, 965, 970, 975, 980, 985, 990, 1015, 1020,
        1025, 1030, 1035, 1040, 1045, 1050, 1055, 1060, 1065, 1070, 1075, 1080, 1085, 1095, 1105, 1115, 1125, 1135, 1145, 1150, 
        1155, 1160, 1165, 1170, 1175, 1180, 1185, 1195, 1205, 1215, 1220, 1225, 1230]) + 500
#trimmed to 1575 snapshots
redo_early = np.arange(0,501)
more_redo_earlier = np.arange(665, 751) #also wanna redo a bit earlier
more_redo = np.arange(1025, 1081) #also wanna redo a bit earlier

coefs = np.load('coefficients_json_3.npy', allow_pickle=True).item()
coefs_late = np.load('coefficients_json_3_redux.npy', allow_pickle=True).item()
coefs_early = np.load('coefficients_json_3_redux_early.npy', allow_pickle=True).item()
coefs_early_2 = np.load('coefficients_json_3_redux_earlier_more.npy', allow_pickle=True).item() ### check
coefs_late_2 = np.load('coefficients_json_3_redux_later_more.npy', allow_pickle=True).item()
snaps = np.sort(glob.glob('json_plots/json_plots/*.png'))
name_list = []
coef_list_loc = np.array([])
i = -1
for snap in snaps: #read in coefs
    i += 1
    name = ((snap.split('json_plots/json_plots/')[1]).split('.png')[0])
    if i <= 1575:
        if i % 5 == 0:
            #print(name)
            name_list.append(name)
            if i in redo_early:
                #print('i in early', i)
                coef_list_loc = np.append(coef_list_loc, 1)
            elif i in redo:
                #print('i in redo', i)
                coef_list_loc = np.append(coef_list_loc, 2)
            elif i in more_redo:
                coef_list_loc = np.append(coef_list_loc, 4)
            elif i in more_redo_earlier:
                coef_list_loc = np.append(coef_list_loc, 5)
            else:
                coef_list_loc = np.append(coef_list_loc, 3)

In [459]:
note_dicts = []
note_name_dict = {}
i = -1
for image in name_list:
    i += 1
    if coef_list_loc[i] == 1:
        sin_cos = coefs_early[image+'_sine']
        cos_cos = coefs_early[image+'_cosine']        
    elif coef_list_loc[i] == 2:
        sin_cos = coefs_late[image+'_sine']
        cos_cos = coefs_late[image+'_cosine']
    elif coef_list_loc[i] == 4:
        sin_cos = coefs_late_2[image+'_sine']
        cos_cos = coefs_late_2[image+'_cosine']
    elif coef_list_loc[i] == 5:
        sin_cos = coefs_early_2[image+'_sine']
        cos_cos = coefs_early_2[image+'_cosine']
    elif coef_list_loc[i] == 3:
        sin_cos = coefs[image+'_sine']
        cos_cos = coefs[image+'_cosine']


    #want to try to trigger events per snap...?
    #print(coef_list_loc[i])
    chord = mod_map_to_sounds_octave_ampcutoff(cos_cos, sin_cos)
    #note_dicts.append([list(chord.keys())][0][::-1]) #flipping voicing?
    #note_name_dict[str(image)] = [list(chord.keys())][0][::-1]
    note_dicts.append([list(chord.keys())][0]) #flipping voicing?
    note_name_dict[str(image)] = [list(chord.keys())][0]

note_set = [list(note_dicts[i]) for i in range(len(note_dicts))]


returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set of notes
returning full set o

In [7]:
#len(note_set), 1575/5, # 195 duration units, 120 bpm, 500000 microseconds per quarter note
(500000*10**(-6))*4, (120/60) # midi tempo = 120 bpm, midi ticks per beat = 480
(500000 * 1e-6 / 480)*191, mid.ticks_per_beat
1560/25

62.4

In [460]:
#note_set[110], map_notes_to_midi_num(note_set[110])#, map_notes_to_midi_num(['D1']), map_notes_to_midi_num(['G1'])
#map_notes_to_midi_num(['D#1','A#1','F1','C1','G1', 'D1', 'A1', 'E1','B1','F#1','C#1','G#1'])
#### code from mike to do this in mido + pyfluidsynth
def mike_remapcoefficients(cos, sin,minlog=0.,maxlog=4.):
    logcoefamplitudes = np.log10(np.abs(cos**2+sin**2))
    # override the max
    maxlog = np.nanmax(logcoefamplitudes)
    logcoefamplitudes = (logcoefamplitudes-minlog)/(maxlog-minlog)*127.
    logcoefamplitudes[logcoefamplitudes<0]=0
    return logcoefamplitudes


# major scale
scale_notes = np.array([60,62,64,65,67,69,71,72])

def make_note_list(logcoefamplitudes,scale_notes,octaveshift=True):

    notes = []
    for harmonic in range(0,5):
        if octaveshift:
            use_notes = scale_notes + (harmonic-1)*12
        else:
            use_notes = scale_notes
        newnote = [use_notes[x] for x in range(0,len(use_notes))]
        newnote = [(note,logcoefamplitudes[i, harmonic]) for i, note in enumerate(newnote) if logcoefamplitudes[i, harmonic] > 0]
        for note in newnote:
            notes.append(note)

    return notes
#make_note_list(,scale_notes

logcoefamplitudes = mike_remapcoefficients(coefs['json_sim_002505_cosine'], coefs['json_sim_002505_sine'])
cc = make_note_list(logcoefamplitudes.T,scale_notes)
cc#[0]#[0]

[(np.int64(48), np.float64(127.0)),
 (np.int64(50), np.float64(68.28159057370934)),
 (np.int64(52), np.float64(112.47844775742603)),
 (np.int64(53), np.float64(107.8373533335468)),
 (np.int64(55), np.float64(72.68115826252306)),
 (np.int64(57), np.float64(101.40277460982385)),
 (np.int64(59), np.float64(104.58062534168829)),
 (np.int64(60), np.float64(100.76143392147051)),
 (np.int64(60), np.float64(49.513784045992395)),
 (np.int64(62), np.float64(39.72565905575779)),
 (np.int64(64), np.float64(41.31893750828786)),
 (np.int64(65), np.float64(43.607670640543766)),
 (np.int64(67), np.float64(47.592751382079555)),
 (np.int64(69), np.float64(46.61505784502144)),
 (np.int64(71), np.float64(41.319809841167555)),
 (np.int64(72), np.float64(36.99503345785822)),
 (np.int64(72), np.float64(54.167364857133656)),
 (np.int64(74), np.float64(51.70552274846624)),
 (np.int64(76), np.float64(52.53921319850652)),
 (np.int64(77), np.float64(49.82144084128581)),
 (np.int64(79), np.float64(44.7251919452743

In [461]:
from mido import Message, MidiFile, MidiTrack
import mido
def mod_map_to_sounds_octave_ampcutoff(cos_cos, sin_cos):
    note_list = np.array(['D#','A#','F','C','G', 'D', 'A', 'E','B','F#','C#','G#'])
    #np.array(['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']) 
    mmax = cos_cos.shape[0]
    nmax = cos_cos.shape[1]

    modulus = np.sqrt(cos_cos**2 + sin_cos**2)
    
    abs_max = np.max(modulus)
    note_dict = {}
    
    for m in range(mmax):
        octave = m + 1 #hmm. need to bump from m = 0 as lowest to m = 1 bc m = 0 doesn't have all notes (?)
        for n in range(nmax):
            amp = modulus[m,n]
            if abs(amp) >= 0.01*abs_max: #will only play terms with amplitude at least 1% of max
                note = note_list[n] + str(octave)
                note_dict[note] = abs(amp)

    print('returning full set of notes')
    return note_dict

def map_notes_to_midi_num(notes):
    midi_note_list = []
    for note_ in notes:
        note = note_[:-1] #getting rid of octave
        octave = int(note_[-1])
        if note == 'C':
            midi_note = 24
        elif note == 'C#':
            midi_note = 25
        elif note == 'D':
            midi_note = 26
        elif note == 'D#':
            midi_note = 27
        elif note == 'E':
            midi_note = 28
        elif note == 'F':
            midi_note = 29
        elif note == 'F#':
            midi_note = 30
        elif note == 'G':
            midi_note = 31
        elif note == 'G#':
            midi_note = 32
        elif note == 'A':
            midi_note = 33
        elif note == 'A#':
            midi_note = 34
        elif note == 'B':
            midi_note = 35
        if octave == 1:
            midi_note_list.append(midi_note)
        else:
            midi_note_list.append(midi_note + octave*12)
    return midi_note_list



def add_chord(track, notes, duration, velocity=127): #64):
    for note in notes:
        track.append(Message('note_on', note=note, velocity=velocity, time=0))
    track.append(Message('note_off', note=notes[0], velocity=velocity, time=duration))# * 480))
    for note in notes[1:]:
        track.append(Message('note_off', note=note, velocity=velocity, time=0))

'''
def make_midi_chords(note_names, chord_length, outputfile):

    # Create a new MIDI file and track
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set the tempo (optional, here 120 BPM)
    tempo = mido.bpm2tempo(120)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))

    # Insert program change at the beginning of the track
    program_change = mido.Message('program_change', program=1, time=0) 
    track.insert(0, program_change)

    notes = map_notes_to_midi_num(note_names)
    add_chord(track, notes, chord_length)

    # Save the MIDI file
    mid.save(outputfile)
'''

def add_chord_stacked(track, notes, duration, i, velocity=127): #64):
    dlist = []
    for note in notes:
        d = duration
        if note > 47:
            d = duration * 2
        if note > 59:
            d = duration * 3
        if note > 61:
            d = duration * 4
        dlist.append(d)
        track.append(Message('note_on', note=note, velocity=velocity, time=0))
    for d in dlist:
        track.append(Message('note_off', note=notes[0], velocity=velocity, time=d))# * 480))
    for note in notes[1:]:
        track.append(Message('note_off', note=note, velocity=velocity, time=0))
                     
        
def make_midi_chords_stacked(note_names, chord_length, outputfile):

    # Create a new MIDI file and track
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set the tempo (optional, here 120 BPM)
    tempo = mido.bpm2tempo(120)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))

    # Insert program change at the beginning of the track
    program_change = mido.Message('program_change', program=1, time=0) 
    track.insert(0, program_change)
    i = -1
    for chord in note_names:
        i += 1
        notes = map_notes_to_midi_num(chord)
        add_chord_stacked(track, notes, i, chord_length)
    # Save the MIDI file
    print('length', mid.length)
    mid.save(outputfile)

def make_midi_chords(note_names, chord_length, outputfile):

    # Create a new MIDI file and track
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set the tempo (optional, here 120 BPM)
    tempo = mido.bpm2tempo(120)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))

    # Insert program change at the beginning of the track
    program_change = mido.Message('program_change', program=1, time=0) 
    track.insert(0, program_change)
    for chord in note_names:
        notes = map_notes_to_midi_num(chord)
        add_chord(track, notes, chord_length)
    # Save the MIDI file
    print('length', mid.length)
    mid.save(outputfile)

In [1]:
#for i in range(10):
#    make_midi_chords(note_set[i], 250, 'try'+str(i)+'.mid') #185
#make_midi_chords(note_set, 191, 'try'+str(33)+'.mid') #185, 195
#! fluidsynth "op1_synth8_try2.sf2" -F "try33.wav" try33.mid

In [363]:
'''
! fluidsynth "op1_synth8_try2.sf2" -F "try1.wav" try1.mid
! fluidsynth "op1_synth8_try2.sf2" -F "try2.wav" try2.mid
! fluidsynth "op1_synth8_try2.sf2" -F "try3.wav" try3.mid
! fluidsynth "op1_synth8_try2.sf2" -F "try4.wav" try4.mid
! fluidsynth "op1_synth8_try2.sf2" -F "try5.wav" try5.mid
'''

'\n! fluidsynth "op1_synth8_try2.sf2" -F "try1.wav" try1.mid\n! fluidsynth "op1_synth8_try2.sf2" -F "try2.wav" try2.mid\n! fluidsynth "op1_synth8_try2.sf2" -F "try3.wav" try3.mid\n! fluidsynth "op1_synth8_try2.sf2" -F "try4.wav" try4.mid\n! fluidsynth "op1_synth8_try2.sf2" -F "try5.wav" try5.mid\n'

In [2]:
#make_midi_chords(note_set[0:10], 2000, 'try1.mid')
#! fluidsynth "op1_synth8_try2.sf2" -F "try1.wav" try1.mid

In [304]:
make_midi_chords(note_set[0:10], 5000, 'try2.mid')
! fluidsynth "op1_synth8_try2.sf2" -F "try2.wav" try2.mid

length 5.208333333333333
FluidSynth runtime version 2.4.6
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
Rendering audio to file 'try2.wav'..
fluidsynth: warning: Instrument not found on channel 0 [bank=0 prog=1], substituted [bank=0 prog=0]


In [41]:
#25 frames per second
# Function to generate video
def generate_video():
    image_folder = './json_plots/json_plots/'
    video_name = 'new_json_sim.avi'

    images = [img.split('/json_plots/json_plots/')[1] for img in np.sort(glob.glob('./json_plots/json_plots/*.png'))]
    print("Images:", images)

    # Set frame from the first image
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # Video writer to create .avi file
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc('a','v','c','1'), 25, (width, height)) #'a','v','c','1'

    # Appending images to video
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Release the video file
    video.release()
    cv2.destroyAllWindows()
    print("Video generated successfully!")

# Calling the function to generate the video
generate_video() 

Images: ['json_sim_000005.png', 'json_sim_000010.png', 'json_sim_000015.png', 'json_sim_000020.png', 'json_sim_000025.png', 'json_sim_000030.png', 'json_sim_000035.png', 'json_sim_000040.png', 'json_sim_000045.png', 'json_sim_000050.png', 'json_sim_000055.png', 'json_sim_000060.png', 'json_sim_000065.png', 'json_sim_000070.png', 'json_sim_000075.png', 'json_sim_000080.png', 'json_sim_000085.png', 'json_sim_000090.png', 'json_sim_000095.png', 'json_sim_000100.png', 'json_sim_000105.png', 'json_sim_000110.png', 'json_sim_000115.png', 'json_sim_000120.png', 'json_sim_000125.png', 'json_sim_000130.png', 'json_sim_000135.png', 'json_sim_000140.png', 'json_sim_000145.png', 'json_sim_000150.png', 'json_sim_000155.png', 'json_sim_000160.png', 'json_sim_000165.png', 'json_sim_000170.png', 'json_sim_000175.png', 'json_sim_000180.png', 'json_sim_000185.png', 'json_sim_000190.png', 'json_sim_000195.png', 'json_sim_000200.png', 'json_sim_000205.png', 'json_sim_000210.png', 'json_sim_000215.png', 'j

import moviepy
#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
videoclip = moviepy.VideoFileClip("json_no_axis.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/try1.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux.mp4",audio_codec="aac", fps=24)

In [9]:
len(np.sort(glob.glob('./json_plots/json_plots/*.png')))

1560

In [8]:
#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
import moviepy
videoclip = moviepy.VideoFileClip("json_highres.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/calmvibes_synth_galaxy.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_calmvibes_highres.mp4",audio_codec="aac", fps=25) #"

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [15080, 14984], 'bitrate': 697608, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.101 libx264'}}], 'input_number': 0}], 'duration': 63.0, 'bitrate': 697611, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [15080, 14984], 'video_bitrate': 697608, 'video_fps': 25.0, 'video_duration': 63.0, 'video_n_frames': 1575}
/Users/cfilion/Developer/sonification/venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -i json_highres.mp4 -loglevel error -f

MoviePy - Done.
MoviePy - Writing video stacked_redux_calmvibes_highres.mp4



MoviePy - Done !
MoviePy - video ready stacked_redux_calmvibes_highres.mp4


In [13]:
#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
import moviepy
videoclip = moviepy.VideoFileClip("json_highres.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/wood_drum_galaxy.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_wood_highres.mp4",audio_codec="aac", fps=25)

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [15080, 14984], 'bitrate': 697608, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.101 libx264'}}], 'input_number': 0}], 'duration': 63.0, 'bitrate': 697611, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [15080, 14984], 'video_bitrate': 697608, 'video_fps': 25.0, 'video_duration': 63.0, 'video_n_frames': 1575}
/Users/cfilion/Developer/sonification/venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -i json_highres.mp4 -loglevel error -f

MoviePy - Done.
MoviePy - Writing video stacked_redux_wood_highres.mp4



MoviePy - Done !
MoviePy - video ready stacked_redux_wood_highres.mp4


#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
videoclip = moviepy.VideoFileClip("json_no_axis.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/wood_drum_galaxy.mp3", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_wood.mp4",audio_codec="aac", fps=24) #"

#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
videoclip = moviepy.VideoFileClip("json_no_axis.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/dune_synth_galaxy.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_dune.mp4",audio_codec="aac", fps=24) #"

In [12]:
#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
videoclip = moviepy.VideoFileClip("json_highres.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/80s_synth_galaxy.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_80s_highres.mp4",audio_codec="aac", fps=25)

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [15080, 14984], 'bitrate': 697608, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.101 libx264'}}], 'input_number': 0}], 'duration': 63.0, 'bitrate': 697611, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [15080, 14984], 'video_bitrate': 697608, 'video_fps': 25.0, 'video_duration': 63.0, 'video_n_frames': 1575}
/Users/cfilion/Developer/sonification/venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -i json_highres.mp4 -loglevel error -f

MoviePy - Done.
MoviePy - Writing video stacked_redux_80s_highres.mp4



MoviePy - Done !
MoviePy - video ready stacked_redux_80s_highres.mp4


In [426]:
#from moviepy.editor import concatenate_audioclips, AudioFileClip
#clip = [moviepy.AudioFileClip(i,fps=44100) for i in audio_clip_paths]
videoclip = moviepy.VideoFileClip("json_no_axis.mp4")
audioclip = moviepy.AudioFileClip("/Users/cfilion/Developer/sonification/calmvibes_synth_galaxy.wav", fps=44100)

videoclip.audio = audioclip
videoclip.write_videofile("stacked_redux_calm.mp4",audio_codec="aac", fps=24) #"

frame_index:  29%|████████████████████████▏                                                          | 558/1919 [1:03:00<00:19, 68.28it/s, now=None]

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [534, 530], 'bitrate': 5273, 'fps': 25.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.101 mpeg4'}}], 'input_number': 0}], 'duration': 63.0, 'bitrate': 5274, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [534, 530], 'video_bitrate': 5273, 'video_fps': 25.0, 'video_duration': 63.0, 'video_n_frames': 1575}
/Users/cfilion/Developer/sonification/venv/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -i json_no_axis.mp4 -loglevel error 


frame_index:  29%|████████████████████████▏                                                          | 558/1919 [1:03:01<00:19, 68.28it/s, now=None]

MoviePy - Done.
MoviePy - Writing video stacked_redux_calm.mp4




frame_index:  29%|████████████████████████▏                                                          | 558/1919 [1:03:02<00:19, 68.28it/s, now=None]

MoviePy - Done !
MoviePy - video ready stacked_redux_calm.mp4


In [397]:
im_list = np.sort(glob.glob('./json_plots/trimmed_json_plots/*.png'))
im_list[1575]

np.str_('./json_plots/trimmed_json_plots/json_sim_007885.png')

In [364]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

# Set the tempo (optional, here 120 BPM)
tempo = mido.bpm2tempo(120)
track.append(mido.MetaMessage('set_tempo', tempo=tempo))

# Insert program change at the beginning of the track
program_change = mido.Message('program_change', program=1, time=0) 
track.insert(0, program_change)
notes = [40, 30, 25, 42]
velocity = 127
duration = 195
i = 0
track.append(Message('note_on', note=27, velocity=velocity, time=0))
track.append(Message('note_on', note=50, velocity=velocity, time=0))# * 480))
track.append(Message('note_on', note=100, velocity=velocity, time=0))
track.append(Message('note_off', note=27, velocity=velocity, time=1000))
track.append(Message('note_off', note=50, velocity=velocity, time=500))# * 480))
track.append(Message('note_off', note=100, velocity=velocity, time=500))

mid.save('try1.mid')

In [366]:
! fluidsynth "op1_synth8_try2.sf2" -F "try1.wav" try1.mid
track

FluidSynth runtime version 2.4.6
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
Rendering audio to file 'try1.wav'..
fluidsynth: warning: Instrument not found on channel 0 [bank=0 prog=1], substituted [bank=0 prog=0]


MidiTrack([
  Message('program_change', channel=0, program=1, time=0),
  MetaMessage('set_tempo', tempo=500000, time=0),
  Message('note_on', channel=0, note=27, velocity=127, time=0),
  Message('note_on', channel=0, note=50, velocity=127, time=0),
  Message('note_on', channel=0, note=100, velocity=127, time=0),
  Message('note_off', channel=0, note=27, velocity=127, time=1000),
  Message('note_off', channel=0, note=50, velocity=127, time=500),
  Message('note_off', channel=0, note=100, velocity=127, time=500)])

# testing strauss again

In [285]:
import matplotlib.pyplot as plt
import strauss
from strauss.sonification import Sonification
from strauss.sources import Objects
from strauss.generator import Generator
from strauss import channels
from strauss.generator import Sampler
from strauss.score import Score
from strauss.generator import Synthesizer
import IPython.display as ipd
from strauss.sources import Events
import os
from scipy.interpolate import interp1d
import numpy as np
import copy
import wavio as wav

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import time

In [328]:
test = Sampler("op1_synth8_try2.sf2", sf_preset=0)
test_generator = copy.copy(test)

#fuck ok so it just.... yolos? what is happening with this, why are some of the chords higher than others even though
#they should all be the same?

for i in range(10):
    note_dict = note_set[i]
    print(note_dict)
    score = Score([note_dict], .4)
    data = {'pitch': np.ones(len(note_dict)),
           'volume': np.ones(len(note_dict))} 
    sources = Events(data.keys())
    sources.fromdict(data)
    lims = {'time': ('0%','105%')}
    sources.apply_mapping_functions(map_lims=lims)
    
    system = "mono"
    soni = Sonification(score, sources, test_generator, system)
    soni.render()
    #soni.save('try2.wav')
    soni.save('try'+str(i)+'.wav')

['G#1', 'C#1', 'F#1', 'B1', 'E1', 'A1', 'D1', 'G1', 'C1', 'F1', 'A#1', 'D#1']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1381.26it/s]


Saved try0.wav
['G#1', 'C#1', 'F#1', 'B1', 'E1', 'A1', 'D1', 'G1', 'C1', 'F1', 'A#1', 'D#1']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1216.06it/s]


Saved try1.wav
['G#1', 'C#1', 'F#1', 'B1', 'E1', 'A1', 'D1', 'G1', 'C1', 'F1', 'A#1', 'D#1']


...

Saved try7.wav
['G#1', 'C#1', 'F#1', 'B1', 'E1', 'A1', 'D1', 'G1', 'C1', 'F1', 'A#1', 'D#1']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1194.39it/s]


Saved try8.wav
['G#1', 'C#1', 'F#1', 'B1', 'E1', 'A1', 'D1', 'G1', 'C1', 'F1', 'D#1']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 1057.10it/s]

Saved try9.wav
